# Explore Fastdb class

In [ ]:
from IPython.display import display, HTML 

In [ ]:
display(HTML("<style>.container { width:90% !important; }</style>"))

## Why

So far I have created two demos, a simple one on `fastdebug.utils.whatinside` and a real world one on `fastcore.meta.FixSigMeta`. They should be enough as demos to show the usefulness of fastdebug library. 

However, I want to be thorough and really prove to myself that it can help me to conquer unknowns and clear doubts with ease and joy when exploring source code. 

Fastdb class and the funcs it uses contains all the tricks I learnt and difficulties I conquered which I don't always remember how and why. In fact I don't want to remember them. I want to just run all cells here and skim through the notebook and everything makes sense to me. 

Can fastdebug help me achieve it? Let's give a try!

## How to do it?

I need a few examples first. Maybe I could just use the simple demo as example for `Fastdb`.

## `whatinside` and `gw['whatinside']` are the same

In [ ]:
from fastdebug.utils import *
import inspect

In [ ]:
gw = {}
gw.update(whatinside.__globals__)
len(gw)

20

In [ ]:
whatinside.__code__.__repr__()

'<code object whatinside at 0x10809da80, file "/Users/Natsume/Documents/fastdebug/fastdebug/utils.py", line 14>'

In [ ]:
gw['whatinside'].__code__

<code object whatinside at 0x10809da80, file "/Users/Natsume/Documents/fastdebug/fastdebug/utils.py", line 14>

In [ ]:
inspect.getsource(gw['whatinside'])

'def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here\n               dun:bool=False, # print all items in __all__\n               func:bool=False, # print all user defined functions\n               clas:bool=False, # print all class objects\n               bltin:bool=False, # print all builtin funcs or methods\n               lib:bool=False, # print all the modules of the library it belongs to\n               cal:bool=False # print all callables\n             ): \n    \'Check what inside a module: `__all__`, functions, classes, builtins, and callables\'\n    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0\n    funcs = inspect.getmembers(mo, inspect.isfunction)\n    classes = inspect.getmembers(mo, inspect.isclass)\n    builtins = inspect.getmembers(mo, inspect.isbuiltin)\n    callables = inspect.getmembers(mo, callable)\n    pkgpath = os.path.dirname(mo.__file__)\n    print(f"{mo.__name__} has: \\n{dun_all} items in its __all__, and \\n{len(funcs)} 

In [ ]:
gw['whatinside'] == whatinside

True

## Dot it in a more natural and ordered way

In [ ]:
from fastdebug.core import *

In [ ]:
import inspect

### Prepare env for Fastdb.dbprint

In [ ]:
g = {}
g.update(Fastdb.dbprint.__globals__)
len(g)

157

In [ ]:
'Fastdb.dbprint' in g

False

In [ ]:
'Fastdb' in g

True

In [ ]:
'dbprint' in g

True

In [ ]:
g['dbprint'] # has nothing, this is probably come from the notebook 00_core

In [ ]:
g['Fastdb'].dbprint

<function fastdebug.core.Fastdb.dbprint(self: fastdebug.core.Fastdb, dbcode: int, cmt: str, *codes, expand: int = 2, showdbsrc: bool = False)>

In [ ]:
inspect.getdoc(g['Fastdb'].dbprint)

'Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating'

In [ ]:
Fastdb.dbprint == g['Fastdb'].dbprint

True

In [ ]:
inspect.getsource(Fastdb.dbprint)

'@patch\ndef dbprint(self:Fastdb, \n            dbcode:int, # a srcline under investigation, can be either string or int\n            cmt:str, # comment added to the srcline\n            *codes, # a list of expressions (str) you write to be evaluated above the srcline\n            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated\n            showdbsrc:bool=False): # display dbsrc\n    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"\n\n    src = self.orisrc\n    if type(dbcode) == int: self.cmts.update({dbcode: cmt})\n\n    printsrc(src, dbcode, cmt, expand)\n\n    dbsrc = ""\n    indent = 4\n    onedbprint = False\n\n    lst = inspect.getsource(src).split(\'\\n\')\n    if not bool(lst[-1]): lst = lst[:-1]\n\n    newlst = []\n    for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes\n        if bool(i.strip()): newlst.append(i)\n    codes

### Create Fastdb object for `Fastdb.dbprint`

In [ ]:
fdb = Fastdb(Fastdb.dbprint, g)

In [ ]:
fdb.print(10, 1)

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # a srcline under investigation, can be either string or int======(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

After running the following line, `Fastdb.dbprint` is updated by exec and source code can't be seen anymore.

In [ ]:
dbsrc = fdb.dbprint(9, "keep original src safe", "self.orisrc", showdbsrc=True)

    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                                                                                                                                                        (8)
    src = self.orisrc===================================================================================================================================(9)
                                                                                                                                       keep original src safe
    if type(dbcode) == int: self.cmts.update({dbcode: cmt})                                                                                             (10)
                                                                                                                                                        (11)
@patch                                                      

As updating `Fastdb.dbprint` with exec will send the updated src to the class `Fastdb`, dbsrc is actually NoneType.

Luckily, we still got `fdb.orisrc` which keep the original `Fastdb.dbprint` for us.

So, we can run `Fastdb.dbprint = fdb.orisrc` to get `Fastdb.dbprint` back to normal

### Make an example with new `Fastdb.dbprint`

In [ ]:
from fastdebug.utils import *

In [ ]:
import fastdebug.core as core

In [ ]:
gw = {}
gw.update(whatinside.__globals__)
len(gw)

20

In [ ]:
fdbw = Fastdb(whatinside, gw) # we are using the updated Fastdb whose dbprint has dbprintinsert() ready

In [ ]:
fdbw.print(10,1)

def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here=============(0)       
               dun:bool=False, # print all items in __all__===============================(1)       
               func:bool=False, # print all user defined functions========================(2)       
               clas:bool=False, # print all class objects=================================(3)       
               bltin:bool=False, # print all builtin funcs or methods=====================(4)       
               lib:bool=False, # print all the modules of the library it belongs to=======(5)       
               cal:bool=False # print all callables=======================================(6)       
             ): ==========================================================================(7)       
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'==(8)       
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0============================(

#### Important lines to run

In [ ]:
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)") # the updated dbprint will run here.
whatinside = new
whatinside(core)
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration



                                                                                            self.orisrc => self.orisrc : <function whatinside at 0x108133310>
             ):                                                                                                                                         (7)
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'                                                                (8)
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0==========================================================================================(9)
                                                                                                                                count num of funcs in __all__
    funcs = inspect.getmembers(mo, inspect.isfunction)                                                                                                  (10)
    classes = inspect.getmembers(mo, inspect.isclass)    

### explore the second srcline of dbprint

In [ ]:
fdb.print(20, 1) # we are using fdb.orisrc to print, so no dbprintinsert() will be seen.

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # a srcline under investigation, can be either string or int======(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

In [ ]:
Fastdb.dbprint = fdb.orisrc
fdb.dbprint(10, "collect and organize cmt by idx", "self.cmts", "dbcode", "cmt", showdbsrc=True)

                                                                                                                                                        (8)
    src = self.orisrc                                                                                                                                   (9)
    if type(dbcode) == int: self.cmts.update({dbcode: cmt})=============================================================================================(10)
                                                                                                                              collect and organize cmt by idx
                                                                                                                                                        (11)
    printsrc(src, dbcode, cmt, expand)                                                                                                                  (12)
@patch                                                     

In [ ]:
# whatinside = fdbw.orisrc
fdbw = Fastdb(whatinside, gw)
fdbw.print(10,1)

def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here=============(0)       
               dun:bool=False, # print all items in __all__===============================(1)       
               func:bool=False, # print all user defined functions========================(2)       
               clas:bool=False, # print all class objects=================================(3)       
               bltin:bool=False, # print all builtin funcs or methods=====================(4)       
               lib:bool=False, # print all the modules of the library it belongs to=======(5)       
               cal:bool=False # print all callables=======================================(6)       
             ): ==========================================================================(7)       
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'==(8)       
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0============================(

In [ ]:
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)



                                                                                                                                  self.cmts => self.cmts : {}


                                                                                                                                         dbcode => dbcode : 9


                                                                                                                   cmt => cmt : count num of funcs in __all__
             ):                                                                                                                                         (7)
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'                                                                (8)
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0==========================================================================================(9)
                                                    

In [ ]:
Fastdb.dbprint.__qualname__

'Fastdb.dbprint'

In [ ]:
Fastdb.dbprint.__qualname__ in g

False

In [ ]:
Fastdb.dbprint.__qualname__.split('.')[0] in g

True

In [ ]:
fdb.orisrc.__qualname__.split('.')[0]

'Fastdb'